# Convolutional Neural Networks
## aka CNN, ConvNet

As a baseline, let's start a lab running with what we already know.

We'll take our deep feed-forward multilayer perceptron network, with ReLU activations and reasonable initializations, and apply it to learning the MNIST digits.

The main part of the code looks like this, we won't run it in the notebook -- it will take too long. Instead, run it in the terminal, where the script is called `mnist-mlp.py`

Note the changes, which are largely about building a classifier instead of a regression model:
* Output layer has one neuron per category, with softmax activation
* Loss function is cross-entropy loss
* Accuracy metric is categorical accuracy

In [ ]:
# imports, setup, load data sets

model = Sequential()
model.add(Dense(20, input_dim=784, kernel_initializer='normal', activation='relu'))
model.add(Dense(15, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_initializer='normal', activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

categorical_labels = to_categorical(y_train, num_classes=10)
start = datetime.datetime.today()

history = model.fit(X_train, categorical_labels, epochs=100, batch_size=100)

# print metrics, plot errors

What are the big takeaways from this experiment?

1. We get pretty impressive "apparent error" accuracy right from the start! A small network gets us to training error 97% by epoch 15 and 98% around epoch 40.
2. The model *appears* to continue to learn, although it does slow down and oscillate a bit.
3. Our test accuracy is about 95% after 100 iterations and 2.5 minutes or so.
4. Therefore, we are overfitting very quickly... most of the "training" turns out to be a waste.
5. For what it's worth, we get 95% accuracy without much work.

This is not terrible compared to other, non-neural-network approaches to the problem. After all, we could probably tweak this a bit and do even better.

But we talked about using deep learning to solve "95%" problems or "98%" problems ... where one error in 20, or 50 simply won't work. If we can get to "multiple nines" of accuracy, then we can do things like automate mail sorting and translation, create cars that react properly (all the time) to street signs, and control systems for robots or drones that function autonomously.

Try two more experiments (try them separately):
1. Add a third, hidden layer.
2. Increase the size of the hidden layers.

Adding another layer slows things down a little (why?) but doesn't seem to make a difference in accuracy.

Adding a lot more neurons into the first topology slows things down significantly -- 10x as many neurons, and only a marginal increase in accuracy. Notice also (in the plot) that the learning clearly degrades after epoch 50 or so.

... We need a new approach!

---

... let's think about this:

### What is layer 2 learning from layer 1? Combinations of pixels

#### Combinations of pixels contain information but...

There are a lot of them (combinations) and they are "fragile" 

In fact, in our last experiment, we basically built a model that memorizes a bunch of "magic" pixel combinations.

What might be a better way to build features?

* When humans perform this task, we look not at arbitrary pixel combinations, but certain geometric patterns -- lines, curves, loops.
* These features are made up of combinations of pixels, but they are far from arbitrary
* We identify these features regardless of translation, rotation, etc.

Is there a way to get the network to do the same thing?

I.e., in layer one, identify pixels. Then in layer 2+, identify abstractions over pixels that are translation-invariant shapes?

We could look at where a "filter" that represents one of these features (e.g., and edge) matches the image.

How would this work?

### Convolution

Convolution in the general mathematical sense is define as follows:

${\begin{aligned}(f*g)(t)&\,{\stackrel {\mathrm {def} }{=}}\ \int _{-\infty }^{\infty }f(\tau )\,g(t-\tau )\,d\tau \\&=\int _{-\infty }^{\infty }f(t-\tau )\,g(\tau )\,d\tau .\end{aligned}}$

The convolution we deal with in deep learning is a simplified case. We want to compare two signals. Here are two visualizations, courtesy of Wikipedia, that help communicate how convolution emphasizes features:

<img src="http://i.imgur.com/EDCaMl2.png" width=500>

---

#### Here's an animation (where we change ${\tau}$) 
<img src="http://i.imgur.com/0BFcnaw.gif">

__In one sense, the convolution captures and quantifies the pattern matching over space__

If we perform this in two dimensions, we can achieve effects like highlighting edges:

<img src="http://i.imgur.com/DKEXIII.png">

The matrix here, also called a convolution kernel, is one of the functions we are convolving. Other convolution kernels can blur, "sharpen," etc. There's a great PDF from Cornell University <a href="bonus/imaging_and_convolution.pdf">in the bonus folder</a>.

### So we'll drop in a number of convolution kernels, and the network will learn where to use them? Nope. Better than that.

## We'll program in the *idea* of discrete convolution, and the network will learn what kernels extract meaningful features!

The values in a (fixed-size) convolution kernel matrix will be variables in our deep learning model. Although inuitively it seems like it would be hard to learn useful params, in fact, since those variables are used repeatedly across the image data, it "focuses" the error on a smallish number of parameters with a lot of influence -- so it should be vastly *less* expensive to train than just a huge fully connected layer like we discussed above.

This idea was developed in the late 1980s, and by 1989, Yann LeCun (at AT&T/Bell Labs) had built a practical high-accuracy system (used in the 1990s for processing handwritten checks and mail).

__How do we hook this into our neural networks?__

* First, we can preserve the geometric properties of our data by "shaping" the vectors as 2D instead of 1D.

* Then we'll create a layer whose value is not just activation applied to weighted sum of inputs, but instead it's the result of a dot-product (element-wise multiply and sum) between the kernel and a patch of the input vector (image).
    * This value will be our "pre-activation" or feed into an activation function

<img src="http://i.imgur.com/ECyi9lL.png">

* If we perform this operation at lots of positions over the image, we'll get lots of outputs, as many as one for every input pixel. 


<img src="http://i.imgur.com/WhOrJ0Y.jpg">

* So we'll add another layer that "picks" the highest convolution pattern match from nearby pixels, which
    * makes our pattern match a little bit translation invariant (a fuzzy location match)
    * reduces the number of outputs significantly
* This layer is commonly called a pooling layer, and if we pick the "maximum match" then it's a "max pooling" layer.

<img src="http://i.imgur.com/9iPpfpb.png">

__The end result is that the kernel or filter together with max pooling creates a value in a subsequent layer which represents the appearance of a pattern in a local area in a prior layer.__

__Again, the network will be given a number of "slots" for these filters and will learn (by minimizing error) what filter values produce meaningful features. This is the key insight into how modern image-recognition networks are able to generalize -- i.e., learn to tell 6s from 7s or cats from dogs.__

<img src="http://i.imgur.com/F8eH3vj.png">

## Ok, let's build our first ConvNet:

(run the script from the console, `mnist-cnn1.py`)

First, we want to explicity shape our data into a 2-D configuration. We'll end up with a 4-D tensor where the first dimension is the training examples, then each example is 28x28 pixels, and we'll explicitly say it's 1-layer deep. (Why? with color images, we typically process over 3 or 4 channels in this last dimension)

In [ ]:
train_libsvm = "../data/mnist"

X_train, y_train = sklearn.datasets.load_svmlight_file(train_libsvm, n_features=784)
X_train = X_train.toarray()
X_train = X_train.reshape( (X_train.shape[0], 28, 28, 1) )
X_train = X_train.astype('float32')
X_train /= 255
y_train = to_categorical(y_train, num_classes=10)

Now the model:

In [ ]:
# setup, imports, load and shape
model = Sequential()

model.add(Conv2D(8, # number of kernels 
                        (4, 4), # kernel size
                        padding='valid',
                        input_shape=(28, 28, 1)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

While that's running, let's look at a number of "famous" convolutional networks!

### LeNet (Yann LeCun, 1998)

<img src="http://i.imgur.com/k5hMtMK.png">

<img src="http://i.imgur.com/ERV9pHW.gif">

### AlexNet (2012)

<img src="http://i.imgur.com/CpokDKV.jpg">

<img src="http://i.imgur.com/Ld2QhXr.jpg">

### Our MNIST ConvNet

In our first convolutional MNIST experiment, we get to 98.75% in 20 epochs (a couple of minutes on CPU)!

The training error is 99.89% so we've almost completely overfit by this point and need to do a little work if we want to keep learning.

Let's add another convolutional layer (script is in `mnist-cnn2.py`):

In [ ]:
model = Sequential()

model.add(Conv2D(8, # number of kernels 
						(4, 4), # kernel size
                        padding='valid',
                        input_shape=(28, 28, 1)))

model.add(Activation('relu'))

model.add(Conv2D(8, (4, 4)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

While that's running, let's look at some more recent ConvNet architectures:

### VGG16 (2014)

<img src="http://i.imgur.com/gl4kZDf.png">

### GoogLeNet (2014)

<img src="http://i.imgur.com/hvmtDqN.png">

*"Inception" layer: parallel convolutions at different resolutions*

<img src="http://i.imgur.com/TCN9C4P.png">

### Residual Networks (2015-)

Skip layers to improve training (error propagation). Residual layers learn from details at multiple previous layers.

<img src="http://i.imgur.com/32g8Ykl.png">

### Back to our labs: Seriously Overfitting

We're making progress on our test error, but just a bit compared to the time, due to the network overfitting the data.

There are a variety of techniques we can take to counter this, discussed earlier. Let's try a relatively simple solution - add a Dropout filter (`mnist-cnn3.py`):

In [ ]:
model = Sequential()

model.add(Conv2D(8, # number of kernels 
						(4, 4), # kernel size
                        padding='valid',
                        input_shape=(28, 28, 1)))

model.add(Activation('relu'))

model.add(Conv2D(8, (4, 4)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Practicalities Part II
## Summary of Optimizer Variants

(Although these techniques are all open and discussed in various research papers, for this section, I credit a great concise blog post summarizing the techniques, by Sebastian Ruder, from which I've excerpted some graphics and content: http://sebastianruder.com/optimizing-gradient-descent/index.html -- great researcher and contributer to the community! Many thanks!)

### Momentum

Keep some (decaying) amount of prior update direction, and mix with current update step, to damp oscillations:

<img src="http://i.imgur.com/XSUHIfm.png" width=600>

### Nesterov Accelerated Gradient

Use our momentum term to "look ahead" at future parameter values and approximate future gradient.
"NAG first makes a big jump in the direction of the previous accumulated gradient (brown vector), measures the gradient and then makes a correction (red vector), which results in the complete NAG update (green vector). This anticipatory update prevents us from going too fast and results in increased responsiveness..."

<img src="http://i.imgur.com/l7Efbf8.png" width=600>

### Adagrad (Adaptive Gradient)

Tune the learning rate automatically based on the gradient. The learning rate is adapted component-wise, and is given by the square root of sum of squares of the historical, component-wise gradient.

"Adagrad's main weakness is its accumulation of the squared gradients in the denominator: Since every added term is positive, the accumulated sum keeps growing during training. This in turn causes the learning rate to shrink and eventually become infinitesimally small, at which point the algorithm is no longer able to acquire additional knowledge."

<img src="http://i.imgur.com/B7eAmPl.gif">

### Adadelta

Adadelta is an extension of Adagrad that seeks to reduce its aggressive, monotonically decreasing learning rate. Instead of accumulating all past squared gradients, Adadelta restricts the window of accumulated past gradients to some fixed size ${w}$.

### *Lab Results -- Looking Good*

The last lab should get you to 99% test error after 15 epochs!

Try one more experiment, with the following changes:

* Make your convolution filters 3x3
* Add a 25% dropout before the Flatten'ed Dense layer
* Use 32 convolution filters in each convolution layer
* Run for 12 epochs

(solution is in `mnist-cnn4.py`)

### RMS Prop

RMSprop is an unpublished, adaptive learning rate method proposed by Geoff Hinton in Lecture 6e of his Coursera Class (http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf).

"RMSprop and Adadelta have both been developed independently around the same time stemming from the need to resolve Adagrad's radically diminishing learning rates. RMSprop in fact is identical to the first update vector of Adadelta ... RMSprop as well divides the learning rate by an exponentially decaying average of squared gradients. Hinton suggests ${\gamma}$ to be set to 0.9, while a good default value for the learning rate ${\eta}$ is 0.001."

### Adam

"Adaptive Moment Estimation (Adam) is another method that computes adaptive learning rates for each parameter. In addition to storing an exponentially decaying average of past squared gradients like Adadelta and RMSprop, Adam also keeps an exponentially decaying average of past gradients similar to momentum."

Retained estimates are the first moment (the mean) and the second moment (the uncentered variance) of the gradients.
Bias-corrected first and second moment estimates are used to update the parameters as in Adadelta and RMSprop.

<img src="http://i.imgur.com/EGZ43p6.gif" width=600>

## *Lab Wrapup*

From the last lab, you should have a test error after 12 epochs of about 0.86%

For one more activity, try changing the optimizer to old-school "sgd" -- just to see how far we've come with these modern gradient descent techniques in the last few years.

About 96.3% test accuracy after 12 epochs. Two key takeaways:

* Without a good optimizer, even a very powerful network design may not achieve results
* In fact, we could replace the word "optimizer" there with
    * initialization
    * activation
    * regularization
    * (etc.)
* All of these elements we've been working with operate together in a complex way to determine final performance
